In [ ]:
import cv2
import numpy as np
import os
import pandas as pd

def find_petri_dish_contour_and_mask(image_path, save_path='./data/'):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    img = cv2.imread(image_path)
    if img is None:
        print(f"錯誤：無法載入圖像 {image_path}")
        return None
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([5, 5, 0])
    upper_yellow = np.array([40, 255, 255])
    mask = cv2.inRange(hsv_img, lower_yellow, upper_yellow)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(img, [max_contour], -1, (0, 255, 0), 2)
    save_contour_path = os.path.join(save_path, 'petri_dish_contour.jpg')
    cv2.imwrite(save_contour_path, img)
    save_mask_path = os.path.join(save_path, 'petri_dish_mask.jpg')
    cv2.imwrite(save_mask_path, mask)
    return save_contour_path, save_mask_path, max_contour

def find_circle_radius(image_path, max_contour, save_path='./data/'):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to load image at {image_path}")
        return None
    mask = np.zeros_like(img, dtype=np.uint8)
    cv2.drawContours(mask, [max_contour], -1, (255, 255, 255), -1)
    img = cv2.bitwise_and(img, mask)
    blur = cv2.GaussianBlur(img, (15, 15), 0)
    cv2.imwrite(os.path.join(save_path, 'blurred.jpg'), blur)
    img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    cv2.imwrite(os.path.join(save_path, 'hsv.jpg'), img_hsv)
    lower_green = np.array([10, 30, 150])
    upper_green = np.array([40, 60, 200])
    mask = cv2.inRange(img_hsv, lower_green, upper_green)
    cv2.imwrite(os.path.join(save_path, 'mask.jpg'), mask)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    cv2.imwrite(os.path.join(save_path, 'opening.jpg'), opening)
    gradient = cv2.morphologyEx(opening, cv2.MORPH_GRADIENT, kernel)
    cv2.imwrite(os.path.join(save_path, 'gradient.jpg'), gradient)
    
    # 找到輪廓
    # # 使用 RETR_EXTERNAL 模式來檢索所有輪廓的層級結構
    # contours, _ = cv2.findContours(gradient, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 使用 RETR_TREE 模式來檢索所有輪廓的層級結構
    contours, hierarchy = cv2.findContours(gradient, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    print(f"找到輪廓數量：{len(contours)}")  # 調試信息
    final_img = img.copy()  # Copy the original image to draw final circles
    
    dish_center, dish_radius = cv2.minEnclosingCircle(max_contour)
    cv2.circle(final_img, (int(dish_center[0]), int(dish_center[1])), int(dish_radius), (0, 255, 0), 2)  # Draw dish
    centers = []
    radii = []
    
    if contours:
        for contour in contours:
            area = cv2.contourArea(contour)
            print(f"輪廓面積：{area}")  # 調試信息
            if area > 1000:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                centers.append((int(x), int(y)))
                radii.append(int(radius))
                cv2.circle(final_img, (int(x), int(y)), int(radius), (255, 0, 0), 2)  # Draw inhibition zones
        cv2.imwrite(os.path.join(save_path, 'circle.jpg'), final_img)
        results = pd.DataFrame({
            'Type': ['Dish'] + ['Inhibition Zone']*len(centers),
            'Center_x': [int(dish_center[0])] + [c[0] for c in centers],
            'Center_y': [int(dish_center[1])] + [c[1] for c in centers],
            'Radius': [int(dish_radius)] + radii
        })
        results.to_csv(os.path.join(save_path, 'CD.csv'), index=False)
        return centers, radii
    return None, None

if __name__ == "__main__":
    image_path = '/home/ubuntu/02.WGS/01.AnyDisc/2023-07-27_11:48:02.jpg'
    save_path = '/home/ubuntu/02.WGS/01.AnyDisc/data/'  
    result_contour_path, result_mask_path, max_contour = find_petri_dish_contour_and_mask(image_path, save_path)
    print(f"輪廓結果已保存至：{result_contour_path}")
    print(f"遮罩結果已保存至：{result_mask_path}")
    centers, radii = find_circle_radius(image_path, max_contour, save_path)
    if centers and radii:
        print(f"圓心及半徑資訊已保存至 {os.path.join(save_path, 'CD.csv')}")



In [ ]:
import os
import cv2
import numpy as np

def Find_circle_radius(image_path):
    save_path = './data/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 讀取圖像
    img = cv2.imread(image_path)
    if img is None:
        print("Error: No image provided")
        return None
    
    # 處理圖像
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([5, 5, 0])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hsv_img, lower_yellow, upper_yellow)
    cv2.imwrite(os.path.join(save_path, 'mask_yellow.jpg'), mask_yellow)

    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found.")
        return None

    max_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(img, dtype=np.uint8)
    cv2.drawContours(mask, [max_contour], -1, (255, 255, 255), -1)
    masked_image = cv2.bitwise_and(img, mask)
    cv2.imwrite(os.path.join(save_path, 'masked_image.jpg'), masked_image)

    # 應用高斯模糊並再次轉換至HSV色彩空間
    blur = cv2.GaussianBlur(masked_image, (15, 15), 0)
    img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    lower_green = np.array([10, 20, 150])
    upper_green = np.array([40, 60, 200])
    mask_green = cv2.inRange(img_hsv, lower_green, upper_green)
    cv2.imwrite(os.path.join(save_path, 'mask_green.jpg'), mask_green)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opening = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)
    gradient = cv2.morphologyEx(opening, cv2.MORPH_GRADIENT, kernel)
    cv2.imwrite(os.path.join(save_path, 'gradient.jpg'), gradient)

    contours, _ = cv2.findContours(gradient, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final_image = img.copy()  # Copy the original image to draw final circles

    if contours:
        for contour in contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if cv2.contourArea(contour) > 1000:
                cv2.circle(final_image, (int(x), int(y)), int(radius), (255, 0, 0), 2)  # Draw circle on final image

    cv2.imwrite(os.path.join(save_path, 'circle.jpg'), final_image)  # Save the final image with circles drawn

    Central_x, Central_y, Radius, width, height, LT_x, LT_y, RB_x, RB_y, Site = [], [], [], [], [], [], [], [], [], []

    if contours:
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            radius = int(w / 2)
            if w > 40:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    Central_x.append(cX)
                    Central_y.append(cY)
                    Radius.append(radius)
                    width.append(w)
                    height.append(h)
                    LT_x.append(x)
                    LT_y.append(y)
                    RB_x.append(x+w)
                    RB_y.append(y+h)
                    Site.append("O" if w > 400 else "I")

    CD_list = pd.DataFrame({
        'Radius': Radius,
        'Central_x': Central_x,
        'Central_y': Central_y,
        'width': width,
        'height': height,
        'Site': Site,
        'lefttop_x': LT_x,
        'lefttop_y': LT_y,
        'rightbottom_x': RB_x,
        'rightbottom_y': RB_y,
    })

    CD_list.to_csv(os.path.join(save_path, 'CD.csv'), index=False)
    CD_list.to_csv('CD.csv', index=False)

    return CD_list
# 主函數進行測試
if __name__ == '__main__':
    test_image_path = '/home/ubuntu/02.WGS/01.AnyDisc/data/References/2024-03-07_11:44:13.jpg'  # 修改為測試圖像的路徑
    result = Find_circle_radius(test_image_path)
    print(result)

In [ ]:
# 算平均HSV
import os
import cv2
import numpy as np

def find_circle_radius_and_print_hsv_average(image_path):
    save_path = './data/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 讀取圖像
    img = cv2.imread(image_path)
    if img is None:
        print("Error: No image provided.")
        return

    # 將圖像從BGR轉換到HSV色彩空間
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([5, 5, 0])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hsv_img, lower_yellow, upper_yellow)
    cv2.imwrite(os.path.join(save_path, 'mask_yellow.jpg'), mask_yellow)

    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found.")
        return

    max_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(img, dtype=np.uint8)
    cv2.drawContours(mask, [max_contour], -1, (255, 255, 255), -1)
    masked_image = cv2.bitwise_and(img, mask)
    masked_hsv = cv2.cvtColor(masked_image, cv2.COLOR_BGR2HSV)
    mean_val = cv2.mean(masked_hsv, mask=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))

    # 直接打印HSV平均值
    print("Average HSV value in the contour: (Hue, Saturation, Value) =", mean_val[:3])

if __name__ == '__main__':
    test_image_path = '/home/ubuntu/02.WGS/01.AnyDisc/data/TestPhoto/2024-03-07_11:44:13.jpg'  # 修改為測試圖像的路徑
    find_circle_radius_and_print_hsv_average(test_image_path)


In [74]:
import os
import cv2
import numpy as np

def Find_circle_radius(image_path):
    save_path = './data/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 讀取圖像
    img = cv2.imread(image_path)
    if img is None:
        print("Error: No image provided")
        return None
    
    # 處理圖像
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([5, 5, 0])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hsv_img, lower_yellow, upper_yellow)
    cv2.imwrite(os.path.join(save_path, 'mask_yellow.jpg'), mask_yellow)

    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found.")
        return None

    max_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(img, dtype=np.uint8)
    cv2.drawContours(mask, [max_contour], -1, (255, 255, 255), -1)
    masked_image = cv2.bitwise_and(img, mask)

    # 計算平均HSV值
    mean_val = cv2.mean(hsv_img, mask=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))
    print("Average HSV value in the contour: (Hue, Saturation, Value) =", mean_val[:3])

    # 根據平均HSV值選擇遮罩參數
    lower_saturation = 30 if mean_val[1] >= 30 else 20
    lower_green = np.array([10, lower_saturation, 150])
    upper_green = np.array([40, 60, 200])

    # 應用高斯模糊並再次轉換至HSV色彩空間
    blur = cv2.GaussianBlur(masked_image, (15, 15), 0)
    img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(img_hsv, lower_green, upper_green)
    cv2.imwrite(os.path.join(save_path, 'mask_green.jpg'), mask_green)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opening = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)
    gradient = cv2.morphologyEx(opening, cv2.MORPH_GRADIENT, kernel)
    cv2.imwrite(os.path.join(save_path, 'gradient.jpg'), gradient)

    contours, _ = cv2.findContours(gradient, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final_image = img.copy()  # Copy the original image to draw final circles

    if contours:
        for contour in contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if cv2.contourArea(contour) > 1000:
                cv2.circle(final_image, (int(x), int(y)), int(radius), (255, 0, 0), 2)  # Draw circle on final image

    cv2.imwrite(os.path.join(save_path, 'circle.jpg'), final_image)  # Save the final image with circles drawn

    Central_x, Central_y, Radius, width, height, LT_x, LT_y, RB_x, RB_y, Site = [], [], [], [], [], [], [], [], [], []

    if contours:
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            radius = int(w / 2)
            if w > 40:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    Central_x.append(cX)
                    Central_y.append(cY)
                    Radius.append(radius)
                    width.append(w)
                    height.append(h)
                    LT_x.append(x)
                    LT_y.append(y)
                    RB_x.append(x+w)
                    RB_y.append(y+h)
                    Site.append("O" if w > 400 else "I")

    CD_list = pd.DataFrame({
        'Radius': Radius,
        'Central_x': Central_x,
        'Central_y': Central_y,
        'width': width,
        'height': height,
        'Site': Site,
        'lefttop_x': LT_x,
        'lefttop_y': LT_y,
        'rightbottom_x': RB_x,
        'rightbottom_y': RB_y,
    })

    CD_list.to_csv(os.path.join(save_path, 'CD.csv'), index=False)
    CD_list.to_csv('CD.csv', index=False)

    return CD_list
# 主函數進行測試
if __name__ == '__main__':
    test_image_path = '/home/ubuntu/02.WGS/01.AnyDisc/static/uploads/2023-10-06_08:56:26.jpg'  # 修改為測試圖像的路徑
    result = Find_circle_radius(test_image_path)
    print(result)

Average HSV value in the contour: (Hue, Saturation, Value) = (27.131816216285987, 49.74876567511924, 183.47509729384353)
   Radius  Central_x  Central_y  width  height Site  lefttop_x  lefttop_y  \
0     222        329        282    444     446    O        151         34   
1     210        321        284    420     435    O        156         40   
2      62        396        347    125     126    I        335        285   
3      57        396        347    115     115    I        340        290   
4      73        335        182    146     132    I        264        120   
5      63        334        182    127     122    I        272        125   

   rightbottom_x  rightbottom_y  
0            595            480  
1            576            475  
2            460            411  
3            455            405  
4            410            252  
5            399            247  


In [93]:
# Best color difference method for Inhibition zone detection
import os
import cv2
import numpy as np

def Find_circle_radius(image_path):
    save_path = './data/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 讀取圖像
    img = cv2.imread(image_path)
    if img is None:
        print("Error: No image provided")
        return None
    
    # 處理圖像
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([5, 5, 0])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hsv_img, lower_yellow, upper_yellow)
    cv2.imwrite(os.path.join(save_path, 'mask_yellow.jpg'), mask_yellow)

    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        print("No contours found.")
        return None

    max_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(img, dtype=np.uint8)
    cv2.drawContours(mask, [max_contour], -1, (255, 255, 255), -1)
    masked_image = cv2.bitwise_and(img, mask)

    # 計算平均HSV值
    mean_val = cv2.mean(hsv_img, mask=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))
    print("Average HSV value in the contour: (Hue, Saturation, Value) =", mean_val[:3])

    # 根據平均HSV值選擇遮罩參數
    lower_hue = max(10, mean_val[0] - 10)
    upper_hue = min(40, mean_val[0] + 10)
    lower_saturation = max(20, mean_val[1] - 10)
    upper_saturation = min(60, mean_val[1] + 20)
    lower_value = max(150, mean_val[2] - 30)
    upper_value = min(200, mean_val[2] + 50)
    
    lower_green = np.array([lower_hue, lower_saturation, lower_value])
    upper_green = np.array([upper_hue, upper_saturation, upper_value])

    # 應用高斯模糊並再次轉換至HSV色彩空間
    blur = cv2.GaussianBlur(masked_image, (15, 15), 0)
    img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(img_hsv, lower_green, upper_green)
    cv2.imwrite(os.path.join(save_path, 'mask_green.jpg'), mask_green)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opening = cv2.morphologyEx(mask_green, cv2.MORPH_OPEN, kernel)
    gradient = cv2.morphologyEx(opening, cv2.MORPH_GRADIENT, kernel)
    cv2.imwrite(os.path.join(save_path, 'gradient.jpg'), gradient)

    contours, _ = cv2.findContours(gradient, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final_image = img.copy()  # Copy the original image to draw final circles

    if contours:
        for contour in contours:
            (x, y), radius = cv2.minEnclosingCircle(contour)
            if cv2.contourArea(contour) > 1000:
                cv2.circle(final_image, (int(x), int(y)), int(radius), (255, 0, 0), 2)  # Draw circle on final image

    cv2.imwrite(os.path.join(save_path, 'circle.jpg'), final_image)  # Save the final image with circles drawn

    Central_x, Central_y, Radius, width, height, LT_x, LT_y, RB_x, RB_y, Site = [], [], [], [], [], [], [], [], [], []

    if contours:
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            radius = int(w / 2)
            if w > 40:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    Central_x.append(cX)
                    Central_y.append(cY)
                    Radius.append(radius)
                    width.append(w)
                    height.append(h)
                    LT_x.append(x)
                    LT_y.append(y)
                    RB_x.append(x+w)
                    RB_y.append(y+h)
                    Site.append("O" if w > 400 else "I")

    CD_list = pd.DataFrame({
        'Radius': Radius,
        'Central_x': Central_x,
        'Central_y': Central_y,
        'width': width,
        'height': height,
        'Site': Site,
        'lefttop_x': LT_x,
        'lefttop_y': LT_y,
        'rightbottom_x': RB_x,
        'rightbottom_y': RB_y,
    })

    CD_list.to_csv(os.path.join(save_path, 'CD.csv'), index=False)
    CD_list.to_csv('CD.csv', index=False)

    return CD_list
# 主函數進行測試
if __name__ == '__main__':
    test_image_path = '/home/ubuntu/02.WGS/01.AnyDisc/data/TestPhoto/2023-08-25_15:52:36.jpg'  # 修改為測試圖像的路徑
    result = Find_circle_radius(test_image_path)
    print(result)

Average HSV value in the contour: (Hue, Saturation, Value) = (25.460850069826716, 29.08962243472924, 182.76175625152646)
    Radius  Central_x  Central_y  width  height Site  lefttop_x  lefttop_y  \
0      225        359        242    450     453    O        135         17   
1      220        357        245    440     443    O        140         22   
2       42        361        371     84      84    I        321        331   
3       37        361        371     74      74    I        326        336   
4       79        462        259    158     166    I        386        182   
5       74        462        258    148     152    I        391        187   
6       89        288        208    178     173    I        196        121   
7       82        290        209    164     161    I        205        128   
8      120        269         84    241     146    I        168         24   
9       50        244         93    101      78    I        188         56   
10      31        373